# Timeseries Sentinel 2 from openEO
In this Notebook, we will extract time series of Sentinel-2 NDVI from a Data Cube for the inundation case

In [2]:
import json

import matplotlib.pyplot as plt
import numpy as np
import openeo
from openeo.rest.job import RESTJob
from openeo.rest.conversions import timeseries_json_to_pandas
import shapely.geometry
import scipy.signal

# Get the data from the backend

In [3]:
# Open the GeoJSON file and read the features
with open("../input/test_site_demervallei_WGS84_v2.geojson") as f:
    features = json.load(f)["features"]

print("GeoJSON Features:", features)

# Assuming a single polygon in the GeoJSON file
geometry = shapely.geometry.shape(features[0]["geometry"])  # Extract the geometry of the first feature

# Get the bounds of the polygon (minx, miny, maxx, maxy)
bounds = geometry.bounds

print("Polygon Geometry:", geometry)
print("Bounds:", bounds)

GeoJSON Features: [{'type': 'Feature', 'properties': {'id': 1}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[4.968710486638257, 51.00869219207143], [4.980305670916424, 51.00784563198833], [4.979613659415014, 51.00357796037619], [4.973508463494545, 51.00378616074288], [4.968512914917515, 51.00519269175937], [4.968710486638257, 51.00869219207143]]]]}}]
Polygon Geometry: MULTIPOLYGON (((4.968710486638257 51.00869219207143, 4.980305670916424 51.00784563198833, 4.979613659415014 51.00357796037619, 4.973508463494545 51.00378616074288, 4.968512914917515 51.00519269175937, 4.968710486638257 51.00869219207143)))
Bounds: (4.968512914917515, 51.00357796037619, 4.980305670916424, 51.00869219207143)


In [4]:
start_date = "2024-05-01"
end_date = "2024-05-08"
bands = [ "B08", "B11", "SCL"]

In [5]:
con  = openeo.connect("https://openeo.vito.be").authenticate_oidc(provider_id="egi")
bbox = {"west": geometry.bounds[0] , "south":geometry.bounds[1] , "east": geometry.bounds[2], "north": geometry.bounds[3], "crs": "EPSG:4326"}
dates = (start_date, end_date)

datacube = con.load_collection("SENTINEL2_L2A", temporal_extent=dates, bands=bands, spatial_extent = bbox)

# Extract band11
band11 = datacube.band("B11")

# Calculate NDMI
ndmi = datacube.ndvi(nir="B08", red="B11")

# Extract the Scene Classification (SCL) layer
classification = datacube.band("SCL")



Authenticated using refresh token.


## Masking with a kernel

In [6]:
# Create a cloud mask using SCL values (4 and 5 are vegetated and non-vegetated)
cloud_mask = ~((classification == 4) | (classification == 5))

# Apply a Gaussian kernel to smooth the mask
g = scipy.signal.windows.gaussian(11, std=1.5)  # 11x11 kernel with std=1.5
kernel = np.outer(g, g)
kernel = kernel / kernel.sum()  # Normalize kernel
cloud_mask = cloud_mask.apply_kernel(kernel)

# Threshold the smoothed mask
cloud_mask = cloud_mask > 0.1  # Threshold to make it binary again


In [8]:
# Apply the mask to both Band 11 and NDMI
masked_band11 = band11.mask(cloud_mask)
masked_ndmi = ndmi.mask(cloud_mask)

## Download the data

In [ ]:
# Save all time slices of Band 11 as individual GeoTiffs
job = masked_band11.save_result(
    format="GTiff",
    options={"tiled": True, "bands_dimension": "t"}  
)

# Execute the batch job
job = job.execute_batch()

# Download the single multiband GeoTIFF file
job_results = job.get_results()
result_files = job_results.download_files(target="../output/band11")

print("GeoTIFF files saved to:", result_files)

0:00:00 Job 'j-2501170806154bb29cf56874dd8e40be': send 'start'
0:00:16 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:00:21 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:00:27 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:00:35 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:00:45 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:00:58 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:01:13 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:01:32 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:01:56 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:02:26 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:03:04 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:03:50 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:04:49 Job 'j-2501170806154bb29cf56874dd8e40be': queued (progress 0%)
0:05:49 Job 'j

In [11]:
# Save all time slices of Band 11 as a multiband GeoTIFF
job = ndmi.save_result(
    format="GTiff",
    options={"tiled": True, "bands_dimension": "t"}  # Combine all time steps into bands
)

# Execute the batch job
job = job.execute_batch()

# Download the single multiband GeoTIFF file
job_results = job.get_results()
result_files = job_results.download_files(target="../output/ndmi")

print("Multiband GeoTIFF file saved to:", result_files)

0:00:00 Job 'j-25011708454640f0af1765162691d5fc': send 'start'
0:00:15 Job 'j-25011708454640f0af1765162691d5fc': queued (progress 0%)
0:00:20 Job 'j-25011708454640f0af1765162691d5fc': queued (progress 0%)
0:00:27 Job 'j-25011708454640f0af1765162691d5fc': queued (progress 0%)
0:00:35 Job 'j-25011708454640f0af1765162691d5fc': queued (progress 0%)
0:00:45 Job 'j-25011708454640f0af1765162691d5fc': queued (progress 0%)
0:00:57 Job 'j-25011708454640f0af1765162691d5fc': queued (progress 0%)
0:01:12 Job 'j-25011708454640f0af1765162691d5fc': queued (progress 0%)
0:01:32 Job 'j-25011708454640f0af1765162691d5fc': queued (progress 0%)
0:01:56 Job 'j-25011708454640f0af1765162691d5fc': running (progress N/A)
0:02:26 Job 'j-25011708454640f0af1765162691d5fc': running (progress N/A)
0:03:03 Job 'j-25011708454640f0af1765162691d5fc': finished (progress 100%)
Multiband GeoTIFF file saved to: [WindowsPath('../output/ndmi/openEO_2024-05-01Z.tif'), WindowsPath('../output/ndmi/openEO_2024-05-04Z.tif'), Window

In [ ]:
# Save the SCL band as a GeoTIFF file
job = classification.save_result(format="GTiff", options={"tiled": True})

# Execute the batch job to process the data and generate the output
job = job.execute_batch()

# Download the resulting GeoTIFF file
job_results = job.get_results()
result_files = job_results.download_files(target="../output/scl_output")

print("SCL GeoTIFF saved to:", result_files)

0:00:00 Job 'j-2501160936344e539d163f22853eb6c9': send 'start'
0:02:13 Job 'j-2501160936344e539d163f22853eb6c9': created (progress 0%)
0:02:18 Job 'j-2501160936344e539d163f22853eb6c9': queued (progress 0%)
0:02:24 Job 'j-2501160936344e539d163f22853eb6c9': queued (progress 0%)
0:02:32 Job 'j-2501160936344e539d163f22853eb6c9': queued (progress 0%)
0:02:42 Job 'j-2501160936344e539d163f22853eb6c9': queued (progress 0%)
0:02:55 Job 'j-2501160936344e539d163f22853eb6c9': finished (progress 100%)
